In [128]:
import requests
import asyncio
import xrpl
import os
from xrpl.wallet import Wallet
from xrpl.models.transactions import OfferCreate, OfferCancel
from xrpl.models.requests import AccountOffers
from xrpl.transaction import submit_and_wait
from xrpl.ledger import get_latest_validated_ledger_sequence
from xrpl.asyncio.clients import AsyncJsonRpcClient
from xrpl.account import get_next_valid_seq_number
from datetime import datetime
import time

In [129]:
wallet_seed = os.getenv("XRPL_ACCOUNT_SECRET")
percent_from_mid=10
volume=10

env='test'

In [130]:
if env=='test':
    client = AsyncJsonRpcClient("https://s.altnet.rippletest.net:51234")
elif env=='prod':
    client = AsyncJsonRpcClient("https://xrplcluster.com")

In [131]:
def get_xrp_price():
    url = "https://api.kraken.com/0/public/Ticker?pair=XRPUSD"
    response = requests.get(url)
    data = response.json()

    #print(data)
    # Extract price from response
    if "error" in data and data["error"]:
        print("Error:", data["error"])
        return None
    
    xrp_price = float(data["result"]["XXRPZUSD"]["a"][0])/2+float(data["result"]["XXRPZUSD"]["b"][0])/2  # "c" contains [last trade price, volume]
    return float(round(xrp_price,4))

async def place_order(seed, taker_gets, taker_pays):

    # Load wallet from the seed
    wallet = Wallet.from_seed(seed, algorithm="secp256k1")

    #print(wallet.classic_address)
    # Get the latest ledger index
    latest_ledger = await asyncio.to_thread(get_latest_validated_ledger_sequence, client)
    sequence = await asyncio.to_thread(get_next_valid_seq_number, wallet.classic_address, client)

    # Create the OfferCreate transaction
    offer_tx = OfferCreate(
        account=wallet.classic_address,
        taker_gets=taker_gets,  # What you are selling
        taker_pays=taker_pays,  # What you want in return
        sequence=sequence,  # Ensure sequence is set
        last_ledger_sequence=latest_ledger + 5  # Valid for the next 5 ledgers
    )

    # Submit the transaction and wait for a result
    response = await asyncio.to_thread(submit_and_wait, offer_tx, client, wallet)

#     # Display transaction response
#     print("\nTransaction Response:")
#     print(response)

    if response.is_successful():
        print(f"Transaction succeeded!")
    

    return response.result['tx_json']['Sequence']

async def check_order_status(seed, offer_sequence):

    # Load wallet from the seed
    wallet = Wallet.from_seed(seed, algorithm="secp256k1")

    # Create the AccountOffers request
    account_offers = AccountOffers(
        account=wallet.classic_address,  # Use wallet address
        ledger_index="validated"
    )

    # Send request to XRPL
    response = await client.request(account_offers)
    result = response.result

    if "offers" in result:
        for offer in result["offers"]:
            if offer["seq"] == offer_sequence:
                print(f"🚀 Order {offer_sequence} is still open.")
                print(f"Remaining Amount: {offer['taker_gets']} | Price: {offer['taker_pays']}")
                return
        print(f"✅ Order {offer_sequence} has been **fully traded** (removed from open offers).")
    else:
        print(f"✅ Order {offer_sequence} has been **fully traded** or never existed.")

    await client.close()

async def cancel_order(wallet_seed, offer_sequence):
   # Load the wallet
    wallet = Wallet.from_seed(wallet_seed, algorithm="secp256k1")  # Ensure correct key type

    # Fetch the correct sequence number
    account_sequence = await asyncio.to_thread(get_next_valid_seq_number, wallet.classic_address, client)

    # Create the OfferCancel transaction
    cancel_tx = OfferCancel(
        account=wallet.classic_address,
        offer_sequence=offer_sequence,
        sequence=account_sequence
    )

    # Submit the transaction and wait for confirmation
    response = await asyncio.to_thread(submit_and_wait, cancel_tx, client, wallet) 
        
async def get_open_orders(seed):

    # Load wallet from the seed
    wallet = Wallet.from_seed(seed, algorithm="secp256k1")

    #print(wallet.classic_address)

    # Create the OfferCreate transaction
    offer_tx = AccountOffers(
                account=wallet.classic_address,
                ledger_index="validated",
    )

    # Submit the transaction and wait for a result
    response = await client.request(offer_tx)

    # Display transaction response
    print("\nOpen Orders:")

    return response

def create_order_payload(price, volume):
    taker_gets = str(volume*1000000)  # Amount in drops (10 XRP = 10,000,000 drops)
    taker_pays = {
        "currency": "USD",
        "issuer": "rvYAfWj5gh67oV6fW32ZzP3Aw4Eubs59B",
        "value": round(volume*price*(1+percent_from_mid/100),4)  # Receiving 10 USD
    }
    
    return taker_gets, taker_pays

In [132]:
last_order_seq=0
multiple_orders=False

while True:
    time.sleep(5)
    now = datetime.now()  # Get current local time
    price = get_xrp_price()
    print('\n\n')
    print(f"{now}: Current XRP price on Kraken: ${price}")
    
    open_order_list=await get_open_orders(wallet_seed)
    
    if open_order_list.is_successful():
        open_orders=open_order_list.result['offers']
        if len(open_orders)==0:
            if last_order_seq==0:
                print('No orders posted yet, we are creating first order!')
                taker_gets, taker_pays=create_order_payload(price, volume)
                last_order_seq=await place_order(wallet_seed, taker_gets, taker_pays)
                print(f'The new order ID is {last_order_seq}!')
            else:
                print(f'The last order ID {last_order_seq} is not present, the transaction was probably fulfilled, closing down!')
                break
                
        elif len(open_orders)==1:
            o=open_orders[0]
            seq=o['seq']
            order_price=1000000*float(o['taker_pays']['value'])/float(o['taker_gets'])
            optimal_price=price*(1+percent_from_mid/100)
            if last_order_seq!=o['seq']:
                print('The ID of last order does not match the one in open_orders, aborting!')
                print(await cancel_order(wallet_seed, o['seq']))
                break
                
            elif abs((optimal_price-order_price)>0.01):
                print(f'The order {seq} price {order_price} is too far from optimal price {optimal_price}, updating order!')
                cancel_order=await cancel_order(wallet_seed, o['seq'])
                if cancel_order.is_successful():
                    print('Order cancellation was successful, placing new order!')
                    taker_gets, taker_pays=create_order_payload(price, volume)
                    last_order_seq=await place_order(wallet_seed, taker_gets, taker_pays)
                    print(f'The new order ID is {last_order_seq}!')                    
                    
                else:
                    print('Order cancellation was not successful, aborting!')
                    print(cancel_order)
                    break
            
            else:
                print(f'The order {seq} price {order_price} is close to the optimal price {optimal_price}, doing nothing!')
            
        else:
            print('Error: More than one order found, cancelling orders and aborting execution!')
            for o in open_orders:
                print(await cancel_order(wallet_seed, o['seq']))
                
            multiple_orders=True
            break
        
    else:
        print('Error: Could not fetch the open orders')
        continue




2025-02-23 20:35:08.882137: Current XRP price on Kraken: $2.5574

Open Orders:
The ID of last order does not match the one in open_orders, aborting!
None
